# Desarrollo 

In [1]:
# Librerias
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import time
print(torch.cuda.is_available())
from torch.utils.data import DataLoader, Subset
import sklearn.model_selection
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import pandas as pd
import torchaudio
from ignite.engine import  Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from codes import utils
from codes import metric as mt
from codes.dataset2 import WetSoundDataset
from sklearn.model_selection import KFold
import torch.nn as nn
from torch.nn.functional import sigmoid
from codes.models import *
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

True


In [2]:
rm -rf ./tmp/

In [3]:
main = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

## 1. Importación y pre-procesamiento del dataset

In [4]:
N_FOLDS = 4
AUDIO_DIR = os.path.join(main,"features/train_set/")
SAMPLE_RATE = 44100
LEN_SEC = 300
LEN_SAMPLES = LEN_SEC*SAMPLE_RATE
NUM_SAMPLES = 1292   #Cantidad de secuencias , no largo de secuencia 12920/40
__class_labels = {
'perro'    : 0,
'rana'     : 1,
'lluvia'   : 2,
'motor'    : 3,
'ave'      : 4
}
N_FFT = 2048
HOP = int(N_FFT/2)
N_MELS = 40
LEN_MBE = round(LEN_SAMPLES/int(HOP*NUM_SAMPLES))
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device {device}")

Using device cuda


#### Dataloaders

In [5]:
wet = WetSoundDataset(AUDIO_DIR,
                        LEN_MBE,
                        NUM_SAMPLES,
                        __class_labels,
                        device)
print(f"There are {len(wet)} samples in the dataset.")

There are 6960 samples in the dataset.


# SEDnet

In [6]:
model = SEDnet(n_class=len(__class_labels))
print(model)

SEDnet(
  (CNN1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(1, 5), stride=(1, 5), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=True)
  )
  (CNN2): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=True)
  )
  (CNN3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=

/home/vpoblete/miniconda3/envs/ia_257/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


## 4. Entrenamiento de la red convolucional

In [7]:
from ignite.engine import  Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage
from ignite.handlers import ModelCheckpoint, EarlyStopping

In [8]:
max_epochs = 500

In [9]:
kfold = KFold(n_splits=N_FOLDS, shuffle=True)
# K-fold Cross Validation model evaluation
%tensorboard --logdir logs/fit
for fold, (train_ids, val_ids) in enumerate(kfold.split(wet)):
    torch.manual_seed(12345) # Inicialización
    # Print
    print(f'FOLD {fold+1}')
    print('--------------------------------')

    kfold_trainset = Subset(wet,train_ids)
    kfold_valset = Subset(wet, val_ids)

    trainloader = DataLoader(
                    kfold_trainset, 
                    batch_size=32, shuffle=True)
    valloader = DataLoader(
                    kfold_valset,
                    batch_size=64,shuffle=False)
    display(len(trainloader),len(valloader))
    
    
    
    model = SEDnet(n_class=len(__class_labels))
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
    criterion = torch.nn.BCELoss(reduction="mean")
    trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
    metrics_torch = {
        'Accuracy':Accuracy(output_transform=mt.thresholded_output_transform),
        'Loss':Loss(criterion,output_transform=mt.thresholded_output_transform),
        'Err':mt.er_rate(output_transform=mt.thresholded_output_transform),
        'F1':mt.f1_score(output_transform=mt.thresholded_output_transform),
    }
    train_evaluator = create_supervised_evaluator(model, metrics=metrics_torch, device=device)
    valid_evaluator = create_supervised_evaluator(model, metrics=metrics_torch, device=device)
    # Contexto de escritura de datos para tensorboard
    #tensorboard --logdir=/tmp/tensorboard
    
    with SummaryWriter(log_dir=f'logs/fit/tensorboard/SEDnet_kfold{fold+1}') as writer:

        @trainer.on(Events.EPOCH_COMPLETED(every=1)) # Cada 1 epocas
        def log_results(engine):
            # Evaluo el conjunto de entrenamiento
            train_evaluator.run(trainloader) 
            writer.add_scalar("train/er", train_evaluator.state.metrics['Err'], engine.state.epoch)
            writer.add_scalar("train/f1", train_evaluator.state.metrics['F1'], engine.state.epoch)
            writer.add_scalar("train/loss", train_evaluator.state.metrics['Loss'], engine.state.epoch)
            writer.add_scalar("train/acc", train_evaluator.state.metrics['Accuracy'], engine.state.epoch)
            
            # Evaluo el conjunto de validación
            valid_evaluator.run(valloader) 
            er = valid_evaluator.state.metrics['Err']
            f1 = valid_evaluator.state.metrics['F1']
            acc = valid_evaluator.state.metrics['Loss']
            loss = valid_evaluator.state.metrics['Accuracy']
            writer.add_scalar("valid/er", er, engine.state.epoch)
            writer.add_scalar("valid/f1", f1, engine.state.epoch)
            writer.add_scalar("valid/loss", acc, engine.state.epoch)
            writer.add_scalar("valid/acc", loss, engine.state.epoch)

        def score_function(engine):
            val_loss = engine.state.metrics['Err']
            return -val_loss

        handler = EarlyStopping(patience=100, score_function=score_function, trainer=trainer)

        # Guardo el mejor modelo en validación
        best_model_handler = ModelCheckpoint(dirname='models/', require_empty=False, filename_prefix="best", n_saved=1,
                                             score_function=lambda engine: -engine.state.metrics['Err'],
                                             score_name="val_error")

        # Lo siguiente se ejecuta cada ves que termine el loop de validación
        valid_evaluator.add_event_handler(Events.COMPLETED, 
                                    best_model_handler, {'SEDnet{}'.format(fold): model})
        valid_evaluator.add_event_handler(Events.COMPLETED, handler)

        trainer.run(trainloader, max_epochs=max_epochs)

FOLD 1
--------------------------------


164

28

/home/vpoblete/miniconda3/envs/ia_257/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/home/vpoblete/miniconda3/envs/ia_257/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
2021-10-12 21:11:17,586 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


FOLD 2
--------------------------------


...

2021-10-13 07:22:37,280 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


FOLD 4
--------------------------------


164

28

2021-10-13 11:27:20,976 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
